In [1]:
import numpy as np
import math
from scipy.stats import gamma
from sklearn.decomposition import IncrementalPCA

import tensorly as tl
from tensorly.cp_tensor import cp_mode_dot
import tensorly.tenalg as tnl
from tensorly.tenalg.core_tenalg import tensor_dot, batched_tensor_dot, outer, inner

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

from pca import PCA

# Import TensorLy
import tensorly as tl
from tensorly.tenalg import kronecker
from tensorly import norm
from tensorly.decomposition import symmetric_parafac_power_iteration as sym_parafac
from tensorly.tenalg.core_tenalg.tensor_product import batched_tensor_dot
from tensorly.testing import assert_array_equal, assert_array_almost_equal

from tensorly.contrib.sparse.cp_tensor import cp_to_tensor

from tlda_final import TLDA
import cumulant_gradient
import tensor_lda_util as tl_util
## Break down into steps, then re-engineer.

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [porter.stem(self.wnl.lemmatize(t,get_wordnet_pos(t))) for t in word_tokenize(articles)]
    
class StemTokenizer(object):
    def __init__(self):
        self.porter = PorterStemmer()
    def __call__(self, articles):
        return [self.porter.stem(t) for t in word_tokenize(articles)]
        
import gc
from datetime import datetime

import scipy

[nltk_data] Downloading package punkt to /home/adrien/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/adrien/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/adrien/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load tweets

import os
from helpers import load_tweets
filenames = ['../data/unzipped/' + name for name in os.listdir('../data/unzipped')]

all_tweets = load_tweets(filenames, preprocessor=None)

In [4]:
import random
# use only 100000 tweets
tweets = random.sample(all_tweets, 100000)

In [5]:
# Import Data and convert to tensor
n_samples = len(tweets)
print(n_samples)
# df         = pd.read_csv("../Data/TwitterSpeech.csv")
# df_p       = pd.read_csv("../Data/paradigm.csv")
# print(df.head())

stop_words = list(stopwords.words('english')) + ["'d", "'ll", "'re", "'s", "'ve", 'hi', "n't"]
# added_words = [#[omitted]]

# stop_words = list(np.append(stop_words,added_words))

100000


In [56]:
help(CountVectorizer)

Help on class CountVectorizer in module sklearn.feature_extraction.text:

class CountVectorizer(_VectorizerMixin, sklearn.base.BaseEstimator)
 |  CountVectorizer(*, input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.int64'>)
 |  
 |  Convert a collection of text documents to a matrix of token counts
 |  
 |  This implementation produces a sparse representation of the counts using
 |  scipy.sparse.csr_matrix.
 |  
 |  If you do not provide an a-priori dictionary and you do not use an analyzer
 |  that does some kind of feature selection then the number of features will
 |  be equal to the vocabulary size found by analyzing the data.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ------

In [6]:
## Pre-process Data
# print(int(0.002*n_samples))
countvec = CountVectorizer(tokenizer=StemTokenizer(),
                                strip_accents = 'unicode', # works 
                                stop_words = stop_words, # works
                                lowercase = True, # works
                                ngram_range = (1,2),
                                max_df = 0.4, # works
                                min_df = int(0.002*n_samples))

dtm = countvec.fit_transform(tweets)
    # df.tweet[ df.year>=2019][:n_samples])

# print(dtm.toarray().shape)

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'ha', 'might', 'must', 'need', 'onc', 'onli', 'ourselv', 'sha', 'themselv', 'thi', 'veri', 'wa', 'whi', 'wo', 'would', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [ ]:
    # find to sentiments
    # sum_words = dtm.sum(axis=0) 
    # words_freq = [(word, sum_words[0, idx]) for word, idx in     countvec.vocabulary_.items()]
    # words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    # print(words_freq[:100])
    
    # top_sents = [ i[0] for i in  words_freq[:1000] if i[0] in df_p["Token"].unique() ]
    # print(top_sents)

In [ ]:
# dtm_pos = np.float_(dtm.toarray())
# dtm_neu = np.float_(dtm.toarray())
# dtm_neg = np.float_(dtm.toarray())
# for i,v in countvec.vocabulary_.items():

#         if i in df_p["Token"].unique() and i in top_sents[:100]:
#             print(i)
#             print(df_p.Positive[df_p.Token==i])
#             dtm_pos[:,v] *= df_p.Positive[df_p.Token==i].unique() 
#             dtm_neg[:,v] *= df_p.Negative[df_p.Token==i].unique() 
#             dtm_neu[:,v] *= df_p.Neutral[df_p.Token==i].unique()
#         else:
#             dtm_pos[:,v] *= 1/3
#             dtm_neg[:,v] *= 1/3
#             dtm_neu[:,v] *= 1/3

In [8]:
# dtm_sent = scipy.sparse.csr_matrix(np.concatenate((dtm_pos,dtm_neg,dtm_neu),axis=1))
dtm_sent = scipy.sparse.csr_matrix(dtm)

In [ ]:
# del dtm_pos,dtm_neg,dtm_neu
# gc.collect()

In [9]:
a = tl.tensor(dtm_sent.toarray(),dtype=np.float16)



In [10]:
del dtm_sent
gc.collect()


537323

In [11]:

M1      = tl.mean(a, axis=0)


In [12]:
x_cent = scipy.sparse.csr_matrix(a - M1,dtype=np.float16) #center the data using the first moment 

In [13]:

gc.collect()


60

In [14]:

start = datetime.now()
print("now =", start)


batch_size = int(n_samples/20)
verbose = True
n_topic =  20

beta_0=0.003

pca = PCA(n_topic, beta_0, 30000)
pca.fit(x_cent) # fits PCA to  data, gives W
x_whit = pca.transform(x_cent) # produces a whitened words counts <W,x> for centered data x
now = datetime.now()
print("now =", now)
pca_time = now - start 

now = 2021-06-30 14:57:23.932368


<ipython-input-14-9e0f7371840d>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  batch_size = np.int(n_samples/20)


now = 2021-06-30 14:58:36.233027


In [15]:

gc.collect()
print(pca_time)

0:01:12.300659


In [18]:
from importlib import reload  
import tlda_final
reload(tlda_final)
from tlda_final import TLDA


now = datetime.now()
print("now =", now)
learning_rate = 0.01 
batch_size =15000
t = TLDA(n_topic,n_senti=1, alpha_0= beta_0, n_iter_train=1000, n_iter_test=150, batch_size=batch_size,
         learning_rate=learning_rate)
now = datetime.now()
print("now =", now)

now = 2021-06-30 15:00:43.186342
now = 2021-06-30 15:00:43.186561


In [19]:
now = datetime.now()
print("now =", now)

t.fit(x_whit,verbose=True) # fit whitened wordcounts to get decomposition of M3 through SGD

now = datetime.now()
print("now =", now)

now = 2021-06-30 15:00:46.878562
Epoch: 200
Epoch: 400
Epoch: 600
Epoch: 800
now = 2021-06-30 15:01:07.775973


In [20]:
now = datetime.now()
print("now =", now)


t.factors_ = pca.reverse_transform(t.factors_)  # unwhiten the eigenvectors to get unscaled word-level factors

''' 
Recover alpha_hat from the eigenvalues of M3
'''  

eig_vals = [np.linalg.norm(k,3) for k in t.factors_ ]
# normalize beta
alpha      = np.power(eig_vals, -2)
print(alpha.shape)
alpha_norm = (alpha / alpha.sum()) * beta_0
t.alpha_   = alpha_norm
        
print(alpha_norm)

t.predict(x_whit,w_mat=True,doc_predict=False)  # normalize the factors 


now = datetime.now()
print("now =", now)

now = 2021-06-30 15:01:36.934330
(20,)
[2.55220743e-04 1.10839107e-04 2.17597749e-04 1.03870983e-04
 6.83356304e-05 6.48767916e-05 1.63810679e-04 1.99996236e-04
 1.13083539e-04 1.22295385e-04 9.84124680e-05 1.02496042e-04
 9.32648939e-05 1.37232622e-04 2.88174135e-04 2.53707756e-04
 6.28712085e-05 1.81468463e-04 9.22223774e-05 2.70223192e-04]
now = 2021-06-30 15:01:36.946814


In [22]:
# ????
# factors= t.factors_
# print((factors.shape))
# factors_reshape = np.concatenate((factors[:,0:(factors.shape[1]//3)],
#                                   factors[:,(factors.shape[1]//3):(2*factors.shape[1]//3)],
#                                   factors[:,(2*factors.shape[1]//3):(factors.shape[1])]),axis=0)
# factors_reshape.shape

In [ ]:
# t.factors_ = factors_reshape
# #t.factors_ = factors
# now = datetime.now()
 
# print("now =", now)

In [24]:

n_top_words=20
#print(t_n_indices)
n_sentiments = 1
for k in range(n_topic*n_sentiments):
    if k ==0:
        t_n_indices   =t.factors_[k,:].argsort()[:-n_top_words - 1:-1]
        top_words_JST = [i for i,v in countvec.vocabulary_.items() if v in t_n_indices]
    else:
        t_n_indices   =t.factors_[k,:].argsort()[:-n_top_words - 1:-1]
        top_words_JST = np.vstack([top_words_JST, [i for i,v in countvec.vocabulary_.items() if v in t_n_indices]])
        print([i for i,v in countvec.vocabulary_.items() if v in t_n_indices])


[',', 'commun', 'partner', 'inform', 'region', 'gt', 'gt ;', ') ,', ', wa', 'public', 'accord', 'high', ', &', 'worker ,', 'includ', ', includ', 'level', 'sars-cov-2', ', may', 'among']
['http http', 'respons', 'covid-19 pandem', 'coronaviru pandem', '. http', 'impact', '–', 'emerg', 'author', 'crisi', 'demand', 'offici', 'amid', 'pandem http', '|', 'amid covid-19', 'ceo', 'u.s.', 'two', 'dure covid-19']
['dure thi', 'thi pandem', 'thank', 'educ', 'worker', 'celebr', 'end', 'frontlin', 'thi #', 'love', 'opportun', "'m", 'happi', 'special', 'especi', 'labour', 'salut', 'surviv', 'thi time', 'intern']
['peopl', '. peopl', ', peopl', 'care', 'risk', 'help', 'protect', 'home', 'die', 'peopl die', 'wear', 'stay home', 'social', 'distanc', 'food', 'back work', 'peopl .', 'social distanc', 'peopl ,', 'mani peopl']
['?', 'dure', 'pandem', 'dure thi', 'thi pandem', '(', ')', 'global', 'pandem ?', '? http', ') http', 'pandem .', 'global pandem', '? ?', 'dure pandem', '? #', 'middl', 'pandem #', 

In [25]:

now = datetime.now()



print("now =", now)
print(t.factors_.shape)
a_word       = tl.tensor(dtm.toarray(),dtype=tl.float32)

doc_topic_dist, topic_word_dist = t.predict(a_word,w_mat=False,doc_predict=True)
now = datetime.now()
 
print("now =", now)
end = datetime.now()
print(end)

now = 2021-06-30 15:04:54.312429
(20, 1615)
now = 2021-06-30 15:11:13.641607
2021-06-30 15:11:13.642524


In [53]:
'hate' in countvec.vocabulary_.keys()

True

In [54]:
countvec.vocabulary_['hate']

768

In [55]:
topic_word_dist[:,768]

array([0.10080967, 0.04867432, 0.00697167, 0.1324299 , 0.13905791,
       0.04894179, 0.01547933, 0.01335646, 0.0314051 , 0.08771742,
       0.08749572, 0.00697167, 0.00697167, 0.08518365, 0.00697167,
       0.07363147, 0.06081808, 0.00697167, 0.02606205, 0.01407878])

In [33]:
doc_topic_dist[8081]

array([5.42159261e-11, 5.42159226e-11, 5.42159347e-11, 6.97614838e-01,
       5.42159606e-11, 5.42227882e-11, 5.42159085e-11, 5.42159523e-11,
       5.42159350e-11, 3.02385161e-01, 5.42159110e-11, 5.42159065e-11,
       5.42159172e-11, 5.42159199e-11, 5.42159289e-11, 5.42159211e-11,
       5.42159153e-11, 5.42159514e-11, 5.42159526e-11, 5.42159204e-11])

In [34]:
tweets[8081]

'The @ONS is launching two online surveys as the organisation steps up its efforts to gather enough #data to help it understand the effects of the #coronavirus on society and the economy.\nRead more: http'

In [ ]:
 from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
dtm_sparse = scipy.sparse.csc_matrix(dtm.toarray(),dtype=np.float16)
dtm_sparse.shape

In [ ]:
start_gensim = datetime.now()
lda = LatentDirichletAllocation(n_components=84,n_jobs=-1,
                                learning_method="online",verbose=1,max_iter=1000,
                                evaluate_every=10,batch_size=15000,max_doc_update_iter=150,perp_tol=1e-3)
lda.fit(dtm)


end_1 = datetime.now()

doc_topic_LDA = lda.transform(dtm)
end_2 = datetime.now()

In [ ]:
print(end-start)
print(end_2-start_gensim)

In [ ]:
n_top_words=20
#print(t_n_indices)

for k in range(n_topic*3):
    if k ==0:
        t_n_indices   =lda.components_[k,:].argsort()[:-n_top_words - 1:-1]
        top_words_LDA = [i for i,v in countvec.vocabulary_.items() if v in t_n_indices]
    else:
        t_n_indices   = lda.components_[k,:].argsort()[:-n_top_words - 1:-1]
        top_words_LDA = np.vstack([top_words_JST, [i for i,v in countvec.vocabulary_.items() if v in t_n_indices]])

In [ ]:
print(top_words_LDA)

In [ ]:
pd.DataFrame(doc_topic_LDA).to_csv("../Data/theta_LDA.csv")

In [ ]:
pd.DataFrame(doc_topic_dist).to_csv("../Data/theta_JST_Tensor.csv")

In [ ]:
pd.DataFrame(top_words_LDA).to_csv("../Data/theta_LDA_TopWords.csv")

In [ ]:
pd.DataFrame(top_words_JST).to_csv("../Data/theta_JST_TopWords.csv")